# Тестовое задание

In [ ]:
import pandas as pd

In [5]:
with open('/Users/krasav4ik_1/Downloads/trial_task.json', 'r') as file:
    data = pd.read_json(file)

In [23]:
df = pd.DataFrame(data)
df

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."
...,...,...,...,...
95,28106,хутор близ Диканьки,-30,"[{'product': 'подписка на suppi-блог', 'price'..."
96,98100,остров невезения,-10,"[{'product': 'автограф Стаса Барецкого', 'pric..."
97,79293,отель Лето,-75,"[{'product': 'статуэтка Ленина', 'price': 200,..."
98,2930,Мордор,-30,"[{'product': 'плюмбус', 'price': 250, 'quantit..."


### Поиск тарифа стоимости доставки для каждого склада

In [17]:
# делим стоимость доставки на общее количество продуктов в заказе, чтобы расчитать тариф
df['total_quantity'] = df['products'].apply(lambda x: sum(i['quantity'] for i in x))
df['tariff'] = df['highway_cost'] / df['total_quantity']

# расчет тарифа для каждого склада
tariff_by_warehouse = df.groupby('warehouse_name')['tariff'].mean()

In [21]:
tariff_by_warehouse

warehouse_name
Мордор                -10.0
гиперборея            -20.0
остров невезения       -5.0
отель Лето            -25.0
хутор близ Диканьки   -15.0
Name: tariff, dtype: float64

### Поиск суммарного количества , суммарного дохода , суммарного расхода и суммарной прибыль для каждого товара.

In [26]:
products = []
for i, row in df.iterrows():
    for product in row['products']:
        product['warehouse_name'] = row['warehouse_name']
        products.append(product)

products_df = pd.DataFrame(products)
products_df['income'] = products_df['price'] * products_df['quantity']

def compute_expense(row):
    return row['quantity'] * tariff_by_warehouse[row['warehouse_name']]

products_df['expenses'] = products_df.apply(compute_expense, axis=1)

products_grouped = products_df.groupby('product').agg({ 
    'quantity': 'sum',                                                      
    'income': 'sum',
    'expenses': 'sum'
}).reset_index()

products_grouped['profit'] = products_grouped['income'] - products_grouped['expenses']

In [27]:
products_grouped

,product,quantity,income,expenses,profit
0,автограф Стаса Барецкого,48,28800,-820.0,29620.0
1,билет в Израиль,58,58000,-1050.0,59050.0
2,зеленая пластинка,61,610,-920.0,1530.0
3,ломтик июльского неба,42,18900,-670.0,19570.0
4,плюмбус,65,16250,-940.0,17190.0
5,подписка на suppi-блог,33,4950,-545.0,5495.0
6,статуэтка Ленина,68,13600,-935.0,14535.0


### Составим табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

In [28]:
df['order_income'] = df['products'].apply(lambda x: sum(i['price'] * i['quantity'] for i in x))
df['order_profit'] = df['order_income'] + df['highway_cost']

order_profit_df = df[['order_id', 'order_profit']]
average_order_profit = order_profit_df['order_profit'].mean()

In [32]:
display(order_profit_df)
print('Средняя прибыль заказов:',average_order_profit)

,order_id,order_profit
0,11973,3980
1,62239,985
2,85794,-30
3,33684,1980
4,25824,975
...,...,...
95,28106,320
96,98100,1590
97,79293,1175
98,2930,720


Средняя прибыль заказов: 1352.3


### Составим табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

In [48]:
df_extended = df.explode('products')

# добавим столбцы для названия продукта, его количества и прибыли.
df_extended['product_name'] = df_extended['products'].apply(lambda x: x['product'])
df_extended['product_quantity'] = df_extended['products'].apply(lambda x: x['quantity'])
df_extended['product_income'] = df_extended['products'].apply(lambda x: x['price'] * x['quantity'])

# сгруппируем и просуммируем. (по началу я посчитал для каждой корзины, но потом решил,
# что не праивльно будет считать как отдельный кейс если совпдает продукт и совпадает склад)
grouped_df = df_extended.groupby(['warehouse_name', 'product_name']).agg({
    'product_quantity': 'sum',
    'product_income': 'sum',
    'highway_cost': 'sum'
}).reset_index()

# добавим столбец с тарифом для каждого склада
grouped_df['tariff'] = (-grouped_df['highway_cost'] / grouped_df['product_quantity']).round(2)

# подсчитаем прибыль
grouped_df['product_profit'] = grouped_df['product_income'] + grouped_df['highway_cost']

#  подсчитаем процент прибыли
warehouse_profit = grouped_df.groupby('warehouse_name')['product_profit'].sum()
grouped_df['percent_profit_product_of_warehouse'] = grouped_df.apply(
    lambda row: (row['product_profit'] / warehouse_profit[row['warehouse_name']]) * 100, axis=1)

# оставим необходимые столбцы
result_df = grouped_df[['warehouse_name', 'product_name', 'product_quantity', 'product_profit', 'percent_profit_product_of_warehouse']]

result_df

,warehouse_name,product_name,product_quantity,product_profit,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,4,2290,13.895631
1,Мордор,билет в Израиль,9,8860,53.762136
2,Мордор,зеленая пластинка,11,-80,-0.485437
3,Мордор,ломтик июльского неба,3,1260,7.645631
4,Мордор,плюмбус,6,1360,8.252427
5,Мордор,подписка на suppi-блог,8,980,5.946602
6,Мордор,статуэтка Ленина,10,1810,10.983010
7,гиперборея,автограф Стаса Барецкого,12,6560,18.344519
8,гиперборея,билет в Израиль,21,20020,55.984340
9,гиперборея,зеленая пластинка,10,-480,-1.342282


###  Табличка с 'accumulated_percent_profit_product_of_warehouse' (накопленный процент).

In [50]:
result_df_sorted = result_df.sort_values(by='percent_profit_product_of_warehouse', ascending=False)
result_df_sorted['accumulated_percent_profit_product_of_warehouse'] =\
                                                result_df_sorted['percent_profit_product_of_warehouse'].cumsum()
result_df_sorted

,warehouse_name,product_name,product_quantity,product_profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
21,отель Лето,билет в Израиль,15,14200,56.349206,56.349206
8,гиперборея,билет в Израиль,21,20020,55.984340,112.333546
1,Мордор,билет в Израиль,9,8860,53.762136,166.095682
27,хутор близ Диканьки,автограф Стаса Барецкого,22,12405,35.821542,201.917224
17,остров невезения,ломтик июльского неба,10,4395,28.781925,230.699150
28,хутор близ Диканьки,билет в Израиль,10,9640,27.837135,258.536285
19,остров невезения,статуэтка Ленина,18,3410,22.331369,280.867654
18,остров невезения,плюмбус,14,3330,21.807466,302.675119
15,остров невезения,билет в Израиль,3,2975,19.482646,322.157765
7,гиперборея,автограф Стаса Барецкого,12,6560,18.344519,340.502284


## Категоризация по накопительному проценту 

In [52]:
def categorize(value):
    if value <= 70:
        return 'A'
    elif value <= 90:
        return 'B'
    else:
        return 'C'

result_df_sorted['category'] = result_df_sorted['accumulated_percent_profit_product_of_warehouse']\
                                                                                    .apply(categorize)
result_df_sorted

,warehouse_name,product_name,product_quantity,product_profit,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
21,отель Лето,билет в Израиль,15,14200,56.349206,56.349206,A
8,гиперборея,билет в Израиль,21,20020,55.984340,112.333546,C
1,Мордор,билет в Израиль,9,8860,53.762136,166.095682,C
27,хутор близ Диканьки,автограф Стаса Барецкого,22,12405,35.821542,201.917224,C
17,остров невезения,ломтик июльского неба,10,4395,28.781925,230.699150,C
28,хутор близ Диканьки,билет в Израиль,10,9640,27.837135,258.536285,C
19,остров невезения,статуэтка Ленина,18,3410,22.331369,280.867654,C
18,остров невезения,плюмбус,14,3330,21.807466,302.675119,C
15,остров невезения,билет в Израиль,3,2975,19.482646,322.157765,C
7,гиперборея,автограф Стаса Барецкого,12,6560,18.344519,340.502284,C
